In [2]:
import os
import glob
import copy
import json
import shutil
from pathlib import Path

import numpy as np
import ansys.fluent.core as pyfluent
from typing import List, Tuple, Dict, Union, Optional, Any, Callable

from utils import WakeConfig, icem_mesh_generation, icem_script_modify
# from visualization import wake_vertical_profile_plot


In [3]:
config = WakeConfig('config_test.yaml')

# ================================= TURBINE ================================
# turbine parameters
rotor_diameter = config.turbine.diameter
hub_height = config.turbine.hub_height

# ================================== MESH ==================================
mesh_folder = config.general.mesh.path

# domain scheme
coord_range = config.mesh.domain_scheme.axis
x_min, x_max = coord_range.x.min, coord_range.x.max
y_min, y_max = coord_range.y.min, coord_range.y.max
z_min, z_max = coord_range.z.min, coord_range.z.max

# ================================ SOLUTION ================================
convge_tol = config.solution.convge_tol

# ================================ SURFACE =================================
# Created surfaces and lines parameters
xy_plane_locs = config.surface.plane.xy_plane
xz_plane_locs = config.surface.plane.xz_plane
yz_plane_locs = config.surface.plane.yz_plane
y_hub_line_locs = config.surface.line.y_axis.x
z_hub_line_locs = config.surface.line.z_axis.x

xy_plane_names = [f'xy_{p}' for p in xy_plane_locs]
xz_plane_names = [f'xz_{p}' for p in xz_plane_locs]
yz_plane_names = [f'yz_{p}d' for p in yz_plane_locs]
y_hub_line_names = [f'y_hub_{p}d' for p in y_hub_line_locs]
z_hub_line_names = [f'z_hub_{p}d' for p in z_hub_line_locs]

# ================================ POSTPROCESS ================================
# default saving path for simulation data, logs, and images
if config.general.run.export:
    image_folder = config.output.image.folder
    data_folder = config.output.data.folder

    # default images and data saving names
    residual_pic_name = config.output.image.residual.name
    ascii_data_name = config.output.data.ascii.name
    tecplot_data_name = config.output.data.tecplot.name
    wake_data_name = config.output.data.field.name

# ================================ PREPARATION ================================
case_path = config.work_dir
if not Path(case_path).is_absolute():
    case_path = Path(case_path).absolute()
if Path.cwd() != case_path:
    os.chdir(case_path)
print(f'Working directory : {case_path}')


Working directory : C:/Users/Li Hang/Documents/Projects/pyfluent/demo


In [4]:
solver_session = pyfluent.launch_fluent(
    mode="solver",
    version='3d',
    precision='double',
    processor_count=6,
    show_gui=True,
    cwd=case_path,
    py=False,
    gpu=False,
    )

if not solver_session.health_check_service.is_serving:
    raise RuntimeError("Fluent is not running normally.")

solver_session.file.cff_files()
solver_session.preferences.General.AutomaticTranscript('no')
solver_session.preferences.General.DefaultIOFormat('Legacy')
solver_session.tui.file.set_batch_options('yes', 'no', 'no')
solver_session.tui.define.user_defined.auto_compile_compiled_udfs('yes')
# solver_session.preferences.Graphics.GridPlaneEnabled = False


Will allow auto-compilation of compiled UDFs as specified in the case or settings file being read.


(This setting will *not* be stored in any case or settings file written.)Note: Rank = 0: Process affinity not being set (6).
Note: Rank = 1: Process affinity not being set (6).
Note: Rank = 2: Process affinity not being set (6).
Note: Rank = 3: Process affinity not being set (6).
Note: Rank = 4: Process affinity not being set (6).
Note: Rank = 5: Process affinity not being set (6).
Reading "C:/Users/Li Hang/Documents/Projects/pyfluent/demo/test.cas"...
Buffering for file scan...



  201828 hexahedral cells, zone  8, binary.
  201828 cell partition ids, zone  8, 6 partitions, binary.
  593329 quadrilateral interior faces, zone  9, binary.
    1452 quadrilateral velocity-inlet faces, zone 10, binary.
    1452 quadrilateral outflow faces, zone 11, binary.
    6116 quadrilateral wall faces, zone 12, binary.
   15290 quadrilateral symmetry faces, zone 13, binary.
  214200 nodes, binary.
214200 node flags, binary.

Checking the existence of compiled "libudf" UDF library for current platform

In [4]:
if config.general.start == 'case':
    rel_case_file_path = Path(config.general.case.path, config.general.case.name + '.cas')
    if not rel_case_file_path.exists():
        raise FileNotFoundError(f'Case file {config.general.case.name}.cas does not exist')
    print(f'Case file to be read: {rel_case_file_path.resolve()}')
    solver_session.file.read_case_data( file_type='case-data', file_name=rel_case_file_path)
elif config.general.start == 'mesh':
    rel_mesh_file_path = Path(config.general.mesh.path, config.general.mesh.name + '.msh')
    if config.general.mesh.from_script:
        rel_mesh_script_path = Path(config.general.mesh.path, config.general.mesh.script)
        print(f'Mesh script to be run first: {rel_mesh_script_path.resolve()}')
        icem_mesh_generation(icem_script_modify(config))
    if not rel_mesh_file_path.exists():
        raise FileNotFoundError(f'Mesh file {config.general.mesh.name}.msh does not exist')
    print(f'Mesh file to be loaded: {rel_mesh_file_path.resolve()}')
    solver_session.file.read_mesh(file_name=rel_mesh_file_path)
else:
    raise ValueError('Simulation should be start from either mesh or case file')

solver_session.mesh.mesh_info()
solver_session.mesh.check()
solver_session.mesh.quality()
solver_session.tui.define.units("length", "m")

if config.general.log.enable:
    config_dict = pyfluent.logging.get_default_config()
    config_dict['handlers']['pyfluent_file']['filename'] = \
        os.oath.join(config.general.log.folder, config.general.log.name + '.txt')
    pyfluent.logging.enable(custom_config=config_dict)
    # pyfluent.logging.set_global_level('DEBUG')

Mesh file to be loaded: C:\Users\Li Hang\Documents\Projects\pyfluent\demo\mesh\test_eq.msh
(This setting will *not* be stored in any case or settings file written.)Note: Rank = 0: Process affinity not being set (6).
Note: Rank = 1: Process affinity not being set (6).
Note: Rank = 2: Process affinity not being set (6).
Note: Rank = 3: Process affinity not being set (6).
Note: Rank = 4: Process affinity not being set (6).
Note: Rank = 5: Process affinity not being set (6).
Reading "mesh\test_eq.msh"...
Buffering for file scan...

  220500 nodes.
  207944 hexahedral cells, zone  8.
  611494 quadrilateral interior faces, zone  9.
    1496 quadrilateral velocity-inlet faces, zone 10.
    1496 quadrilateral pressure-outlet faces, zone 11.
    6116 quadrilateral wall faces, zone 12.
   15568 quadrilateral wall faces, zone 13.

Building...
     mesh
	auto partitioning mesh by Metis (fast),
	distributing mesh
		parts......,
		faces......,
		nodes......,
		cells......,
        bandwidth reductio

In [5]:
if config.general.run.solution:
    solver_session.setup.general.solver.time = 'steady'
    # solver_session.tui.define.models.steady()
    solver_session.setup.general.solver.type = "pressure-based"
    # solver_session.tui.define.models.solver.pressure_based()
    solver_session.setup.models.viscous = {
        'model': 'reynolds-stress',
        'options': None,
        'near_wall_treatment': {'wall_function': 'standard-wall-fn'},
        'reynolds_stress_model': 'rsm-linear-pressure-strain',
        'reynolds_stress_options': {'solve_tke': True, 'wall_echo': True},
        'turbulence_expert': {'turb_non_newtonian': False},
        'user_defined': None
        }

    solver_session.tui.define.user_defined.user_defined_memory(12)
    solver_session.tui.define.user_defined.use_built_in_compiler('yes')

    if config.general.start == 'case':
        solver_session.tui.define.user_defined.compiled_functions('unload', 'libudf')
        default_udflib = os.path.join(os.getcwd(), 'libudf')
        if Path(default_udflib).exists():
            shutil.rmtree(default_udflib, ignore_errors=True)

    inflow_udf = Path(config.udf_file.path, config.udf_file.inflow.name)
    turbine_udf = Path(config.udf_file.path, config.udf_file.turbine.name)

    if config.udf_file.inflow.enable:
        print(f'Inflow UDF to be loaded: {inflow_udf.resolve()}')
    if config.udf_file.turbine.enable:
        print(f'Turbine UDF to be loaded: {turbine_udf.resolve()}')

    solver_session.tui.define.user_defined.compiled_functions(
        'compile', 'libudf', 'yes',
        str(inflow_udf),
        str(turbine_udf),
        '', ''
        )
    solver_session.tui.define.user_defined.compiled_functions('load', 'libudf')


Adjusting the following setting:
Changing Discretization Scheme for Turbulent Kinetic Energy:   from: Second Order Upwind   to: First Order Upwind
Inflow UDF to be loaded: C:\Users\Li Hang\Documents\Projects\pyfluent\demo\inflow_test.c


 Give C-Source file names:

 Give header file names:
Copy ".\adm_test.c" to "libudf\\src"

"mesh\inflow_test.c" is not found Copy "inflow_test.c" to "libudf\\src"


Copyright 1987-2023 ANSYS, Inc. All Rights Reserved.
Compiler and linker: Clang (builtin)
Compiler path: "C:\\PROGRA~1\\ANSYSI~1\\v231\\fluent"\\ntbin\\clang\\bin\\clang-cl
Linker   path: "C:\\PROGRA~1\\ANSYSI~1\\v231\\fluent"\\ntbin\\clang\\bin\\lld-link
C sources: ['adm_test.c', 'inflow_test.c']

Copyright 1987-2023 ANSYS, Inc. All Rights Reserved.
Compiler and linker: Clang (builtin)
Compiler path: "C:\\PROGRA~1\\ANSYSI~1\\v231\\fluent"\\ntbin\\clang\\bin\\clang-cl
Linker   path: "C:\\PROGRA~1\\ANSYSI~1\\v231\\fluent"\\ntbin\\clang\\bin\\lld-link
C sources: ['adm_test.c', 'inflow_test.c

In [6]:
if config.general.run.solution:
    solver_session.setup.models.viscous.model = 'reynolds-stress'
    # solver_session.tui.define.models.viscous.reynolds_stress_model()

    solver_session.setup.materials.fluid['air'] = {
        'density': {'option': 'constant', 'value': 1.225},
        'viscosity': {'option': 'constant', 'value': 1.7894e-05}}
    solver_session.setup.cell_zone_conditions.fluid[
        config.mesh.boundary_condition.flow_name].material = 'air'
    if config.udf_file.inflow.enable:
        # solver_session.setup.cell_zone_conditions.fluid[
        #     config.mesh.boundary_condition.flow_name].sources = False
        # solver_session.setup.cell_zone_conditions.fluid = {
        #     config.mesh.boundary_condition.flow_name: {
        #         'material': 'air',
        #         'sources': True,
        #         'source_terms': {
        #             'x-momentum': [],
        #             'y-momentum': [],
        #             'z-momentum': [],
        #             'k': [{'option': 'udf', 'udf': 'k_source::libudf'}],
        #             'epsilon': [{'option': 'udf', 'udf': 'w_source::libudf'}]
        #             }
        #         }
        #     }

        solver_session.tui.define.boundary_conditions.fluid(
            config.mesh.boundary_condition.flow_name,
            'no', 'yes',
            '0',
            '0', # 'no', 'yes', 'x_source::libudf',
            '0', # 'no', 'yes', 'y_source::libudf',
            '0', # 'no', 'yes', 'z_source::libudf',
            '1', 'no', 'yes', 'k_source::libudf',
            '1', 'no', 'yes', 'w_source::libudf',
            '0', '0', '0', '0', '0', '0', 'no', 'no', 'no', '0', 'no', '0',
            'no', '0', 'no', '0', 'no', '0', 'no', '1', 'no', 'no', 'no', 'no', 'no'
        )

    if config.udf_file.turbine.enable:
        solver_session.setup.cell_zone_conditions.fluid = {
            config.mesh.boundary_condition.flow_name: {
                'material': 'air',
                'sources': True,
                'source_terms': {
                    'x-momentum': [{'option': 'udf', 'udf': 'x_source::libudf'}],
                    'y-momentum': [{'option': 'udf', 'udf': 'y_source::libudf'}],
                    'z-momentum': [{'option': 'udf', 'udf': 'z_source::libudf'}],
                    'k': [{'option': 'udf', 'udf': 'k_source::libudf'}],
                    'epsilon': [{'option': 'udf', 'udf': 'w_source::libudf'}]
                    }
                }
            }

    solver_session.setup.boundary_conditions.change_type(
        zone_list=[config.mesh.boundary_condition.inlet_name],
        new_type='velocity-inlet')
    if config.udf_file.inflow.enable:
        solver_session.setup.boundary_conditions.velocity_inlet = {
            config.mesh.boundary_condition.inlet_name: {
                'velocity_spec': 'Magnitude, Normal to Boundary',
                'frame_of_reference': 'Absolute',
                'vmag': {'option': 'udf', 'udf': 'inlet_velocity::libudf'},
                'initial_gauge_pressure': {'option': 'value', 'value': 0.0},
                'ke_spec': 'K and Epsilon',
                'k': {'option': 'udf', 'udf': 'k_profile::libudf'},
                'e': {'option': 'udf', 'udf': 'w_profile::libudf'},
                'rst_spec': 'K or Turbulent Intensity'
                },
            }

    solver_session.setup.boundary_conditions.change_type(
        zone_list=[config.mesh.boundary_condition.outlet_name],
        new_type='outflow')
    solver_session.setup.boundary_conditions.outflow = {
        config.mesh.boundary_condition.outlet_name: {
            'flowrate_frac': 1}
        }

    # solver_session.setup.boundary_conditions.change_type(
    #     zone_list=[config.mesh.boundary_condition.ground_name],
    #     new_type='wall')
    solver_session.tui.define.boundary_conditions.wall(
        config.mesh.boundary_condition.ground_name,
        'no', 'no', 'no',
        '3e-05', 'no', '0.5'
        )
    solver_session.setup.boundary_conditions.wall = {
        config.mesh.boundary_condition.ground_name: {
            'motion_bc': 'Stationary Wall',
            'shear_bc': 'No Slip',
            'roughness_height': {'option': 'value', 'value': 3e-05},
            'roughness_const': {'option': 'value', 'value': 0.5}}
        }

    solver_session.setup.boundary_conditions.change_type(
        zone_list=[config.mesh.boundary_condition.boundary_name],
        new_type='symmetry')
    solver_session.setup.boundary_conditions.symmetry = {
        config.mesh.boundary_condition.boundary_name: None,
        }


UDM Offset for Current Loaded Library = 0(wind)
The following solver settings object method could also be used to execute the above command:
<solver_session>.setup.cell_zone_conditions.fluid['wind'] = {"sources" : True, "source_terms" : {"mass" : [], "x-momentum" : [], "y-momentum" : [], "z-momentum" : [], "k" : [{"udf" : "k_source::libudf", "option" : "udf"}], "epsilon" : [{"udf" : "w_source::libudf", "option" : "udf"}], "uu-stress" : [], "vv-stress" : [], "ww-stress" : [], "uv-stress" : [], "vw-stress" : [], "uw-stress" : []}}
(ground boundary)
The following solver settings object method could also be used to execute the above command:
<solver_session>.setup.boundary_conditions.wall['ground'] = {"roughness_height" : 3e-05}


In [7]:
solver_session.setup.cell_zone_conditions.fluid()

{'wind': {'material': 'air',
  'sources': True,
  'source_terms': {'epsilon': [{'option': 'udf', 'udf': 'w_source::libudf'}],
   'k': [{'option': 'udf', 'udf': 'k_source::libudf'}],
   'ww-stress': None,
   'vw-stress': None,
   'x-momentum': None,
   'uw-stress': None,
   'y-momentum': None,
   'vv-stress': None,
   'uu-stress': None,
   'z-momentum': None,
   'uv-stress': None,
   'mass': None},
  'fixed': False,
  'mrf_motion': False,
  'reference_frame_axis_origin': [{'option': 'value', 'value': 0.0},
   {'option': 'value', 'value': 0.0},
   {'option': 'value', 'value': 0.0}],
  'reference_frame_axis_direction': [{'option': 'value', 'value': 0},
   {'option': 'value', 'value': 0},
   {'option': 'value', 'value': 1}],
  'laminar': False,
  'porous': False,
  'fanzone': False}}

In [8]:
solver_session.setup.boundary_conditions()

{'interior': {'int_wind': None},
 'outflow': {'outlet': {'flowrate_frac': 1}},
 'symmetry': {'boundary': None},
 'velocity_inlet': {'inlet': {'velocity_spec': 'Magnitude, Normal to Boundary',
   'frame_of_reference': 'Absolute',
   'vmag': {'option': 'udf', 'udf': 'inlet_velocity::libudf'},
   'initial_gauge_pressure': {'option': 'value', 'value': 0},
   'ke_spec': 'K and Epsilon',
   'k': {'option': 'udf', 'udf': 'k_profile::libudf'},
   'e': {'option': 'udf', 'udf': 'w_profile::libudf'},
   'rst_spec': 'K or Turbulent Intensity'}},
 'wall': {'ground': {'motion_bc': 'Stationary Wall',
   'shear_bc': 'No Slip',
   'roughness_height': {'option': 'value', 'value': 3e-05},
   'roughness_const': {'option': 'value', 'value': 0.5}}}}

In [9]:
if config.general.run.solution:
    solver_session.solution.methods.p_v_coupling.flow_scheme = 'SIMPLE'
    solver_session.solution.monitor.convergence_conditions.condition = 'all-conditions-are-met'
    solver_session.tui.solve.monitors.residual.plot("yes")
    solver_session.tui.solve.monitors.residual.criterion_type('0')
    solver_session.tui.solve.monitors.residual.convergence_criteria(
        convge_tol, convge_tol, convge_tol, convge_tol, convge_tol, convge_tol,
        convge_tol, convge_tol, convge_tol, convge_tol, convge_tol, convge_tol
        )

    solver_session.file.confirm_overwrite = True
    solver_session.file.auto_save.retain_most_recent_files = True
    solver_session.file.auto_save.case_frequency = 'if-case-is-modified'
    solver_session.file.auto_save.data_frequency = 0

Copy "C:\Users\Li Hang\Documents\Projects\pyfluent\demo\inflow_YangYi.c" to "libudf\\src"


Copyright 1987-2023 ANSYS, Inc. All Rights Reserved.
Compiler and linker: Clang (builtin)
Compiler path: "C:\\PROGRA~1\\ANSYSI~1\\v231\\fluent"\\ntbin\\clang\\bin\\clang-cl
Linker   path: "C:\\PROGRA~1\\ANSYSI~1\\v231\\fluent"\\ntbin\\clang\\bin\\lld-link
C sources: ['inflow_YangYi.c']

Copyright 1987-2023 ANSYS, Inc. All Rights Reserved.
Compiler and linker: Clang (builtin)
Compiler path: "C:\\PROGRA~1\\ANSYSI~1\\v231\\fluent"\\ntbin\\clang\\bin\\clang-cl
Linker   path: "C:\\PROGRA~1\\ANSYSI~1\\v231\\fluent"\\ntbin\\clang\\bin\\lld-link
C sources: ['inflow_YangYi.c']

Done.

forestriveral: Opening library "C:\\Users\\Li Hang\\Documents\\Projects\\pyfluent\\demo\\libudf"...
Done.

 Standard k-e turbulence model for inflow profile (Yi Yang et al.): u_star= 0.325600, K= 0.418700, rho= 1.225000, z_0= 0.000221C_mu= 0.400000, C_1= -0.976900, C_2= 7.282900Sigma_e= 1.300000, C2e_1e= 0.880000

forestriv

In [ ]:
if config.general.run.solution:
    solver_session.solution.initialization.initialization_type = 'hybrid'  # ['standard', 'hybrid']
    # solver_session.solution.initialization.hybrid_initialize()
    solver_session.solution.initialization.initialize()

    solver_session.tui.solve.set.reporting_interval(config.solution.report_interval)
    solver_session.solution.run_calculation.iterate(iter_count=config.solution.iteration)

 Zone not slit.
Initialize using the hybrid initialization method.

Checking case topology... 
-This case has both inlets & outlets 
-Pressure information is not available at the boundaries.
 Case will be initialized with constant pressure

	iter		scalar-0

	1		1.000000e+00
	2		7.824781e-08
	3		6.229464e-09
	4		3.862875e-10
	5		3.702011e-11
	6		3.052853e-12
	7		2.820815e-13
	8		2.680819e-14
	9		2.366341e-15
	10		2.296197e-16

Hybrid initialization is done.
The following solver settings object method could also be used to execute the above command:
<solver_session>.solution.run_calculation.reporting_interval = 50

  iter  continuity  x-velocity  y-velocity  z-velocity           k     epsilon   uu-stress   vv-stress   ww-stress   uv-stress   vw-stress   uw-stress     time/iter
    50  4.5077e-03  5.9996e-05  5.9384e-06  8.4255e-06  2.3600e-03  3.5843e-03  6.1812e-03  3.3910e-03  2.1896e-03  9.0977e-04  2.8356e-04  2.0882e-03  0:04:14  950
   100  1.9385e-03  2.9716e-05  3.3516e-06  3.447

In [ ]:
if config.output.image.residual.enable:
    solver_session.tui.plot.residuals(
        'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes')
    rel_residual_pic = Path(image_folder, residual_pic_name + '.png')
    if rel_residual_pic.exists(): os.remove(rel_residual_pic)
    solver_session.results.graphics.picture.save_picture(file_name=rel_residual_pic)
    # solver_session.tui.display.save_picture(rel_residual_pic)
    print(f'Residual picture saved to {rel_residual_pic.resolve()}')

Residual picture saved to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\image\run_residual.png


In [10]:
# function to create planes
def create_plane(
    name: str,
    loc: Union[float, int],
    plane: str,
    ) -> None:
    assert isinstance(loc, (float, int)), \
        'Location of plane should be float or int!'
    print(f'Creating plane [{name}] at {loc:.3f} in {plane}...')
    normal_vector = {'xy-plane': 'z', 'zx-plane': 'y', 'yz-plane': 'x'}
    exsiting_planes = solver_session.results.surfaces.plane_surface.keys()
    if name not in exsiting_planes:
        solver_session.results.surfaces.plane_surface.create(name)
    # solver_session.tui.surface.plane_surface(plane_name, 'xy-plane', '0.125')
    solver_session.results.surfaces.plane_surface[name] = {
        'methods': plane,
        normal_vector[plane]: loc
        }

# function to create lines
def create_line(
    name: str,
    loc: Union[float, int],
    pos: Union[float, int]
    ) -> None:
    axis = name.split('_')[0]
    assert isinstance(loc, (float, int)) and isinstance(pos, (float, int)), \
        'Location and position of line should be float or int!'
    print(f'Creating line [{name}] along with {axis}-axis in {loc:.3f} with {pos}...')
    solver_session.tui.surface.delete_surface(name)
    if axis == 'y':
        x_0, y_0, z_0 = loc, y_min, pos
        x_1, y_1, z_1 = loc, y_max, pos
    elif axis == 'z':
        x_0, y_0, z_0 = loc, pos, z_min
        x_1, y_1, z_1 = loc, pos, z_max
    else:
        raise ValueError('axis should be either y or z!')
    solver_session.tui.surface.line_surface(
        name,
        x_0, y_0, z_0, x_1, y_1, z_1
        )

if config.general.run.create:
    # create xy-planes
    for plane_name, plane_loc in zip(xy_plane_names, xy_plane_locs):
        plane_loc = hub_height if plane_loc == 'hub' else plane_loc
        create_plane(plane_name, plane_loc, 'xy-plane')

    # create xz-planes
    for plane_name, plane_loc in zip(xz_plane_names, xz_plane_locs):
        plane_loc = 0.0 if plane_loc == 'hub' else plane_loc
        create_plane(plane_name, plane_loc, 'zx-plane')

    # create yz-planes
    for plane_name, plane_loc in zip(yz_plane_names, yz_plane_locs):
        create_plane(plane_name, plane_loc * rotor_diameter, 'yz-plane')

    # create y-lines in xy-plane
    for line_name, line_loc in zip(y_hub_line_names, y_hub_line_locs):
        create_line(line_name, plane_loc * rotor_diameter, hub_height)

    # create z-lines in xz-plane
    for line_name, line_loc in zip(z_hub_line_names, z_hub_line_locs):
        create_line(line_name, plane_loc * rotor_diameter, 0.0)

    # create z-lines in inflow-plane
    create_line('z_hub_inflow', x_min, 0.0)

Creating plane [xy_hub] at 0.125 in xy-plane...
Creating plane [xz_hub] at 0.000 in zx-plane...
Creating plane [yz_-3d] at -0.450 in yz-plane...
Creating plane [yz_-2d] at -0.300 in yz-plane...
Creating plane [yz_-1d] at -0.150 in yz-plane...
Creating plane [yz_0d] at 0.000 in yz-plane...
Creating plane [yz_1d] at 0.150 in yz-plane...
Creating plane [yz_2d] at 0.300 in yz-plane...
Creating plane [yz_3d] at 0.450 in yz-plane...
Creating plane [yz_4d] at 0.600 in yz-plane...
Creating plane [yz_5d] at 0.750 in yz-plane...
Creating plane [yz_6d] at 0.900 in yz-plane...
Creating plane [yz_7d] at 1.050 in yz-plane...
Creating plane [yz_8d] at 1.200 in yz-plane...
Creating plane [yz_9d] at 1.350 in yz-plane...
Creating plane [yz_10d] at 1.500 in yz-plane...
Creating plane [yz_11d] at 1.650 in yz-plane...
Creating plane [yz_12d] at 1.800 in yz-plane...
Creating plane [yz_13d] at 1.950 in yz-plane...
Creating plane [yz_14d] at 2.100 in yz-plane...
Creating plane [yz_15d] at 2.250 in yz-plane...

In [11]:
solver_session.tui.surface.list_surfaces()

surface group name          id    points  0D facets   1D facets   2D facets
------------------------- ---- --------- ---------- ----------- -----------
boundary                     0         0          0           0           0
ground                       1         0          0           0           0
inlet                        3         0          0           0           0
outlet                       2         0          0           0           0
xy_hub                       4         0          0           0           0
xz_hub                       5         0          0           0           0
y_hub_-1d                   27        45          0          44           0
y_hub_-2d                   26        45          0          44           0
y_hub_-3d                   25        45          0          44           0
y_hub_0d                    28        45          0          44           0
y_hub_10d                   38        45          0          44           0
y_hub_11d   

In [12]:
# function to create contours
def create_contour(name: str) -> None:
    print(f'Creating contour plot figure [{name}]...')
    default_fields = {'vel':'x-velocity', 'turb':'turb-intensity', 'tke':'turb-kinetic-energy'}
    surface = ['_'.join(name.split('_')[:-1])]
    field = default_fields[name.split('_')[-1]]
    exsiting_contour = solver_session.results.graphics.contour.keys()
    if name not in exsiting_contour:
        solver_session.results.graphics.contour.create(name)
    solver_session.results.graphics.contour[name] = {
        'surfaces_list': surface,
        "coloring": {"option": "smooth", "smooth": False},
        "field": field,
        'node_values': True,
        "filled": True,
        'range_option': {'option': 'auto-range-on',
                         'auto_range_on': {'global_range': True}},
        }

    if config.surface.plane.screen_shot.enable:
        solver_session.results.graphics.contour[name].display()
        if name.split('_')[0] == 'xy':
            # solver_session.results.graphics.views.camera.position(right=0., up=-90.)
            # solver_session.results.graphics.views.camera.zoom(factor=1.5)
            pass
        elif name.split('_')[0] == 'xz':
            solver_session.results.graphics.views.camera.orbit(right=0., up=-90.)
            # solver_session.results.graphics.views.camera.zoom(factor=0.5)
        rel_contour_pic = Path(image_folder, name + '.png')
        if rel_contour_pic.exists(): os.remove(rel_contour_pic)
        solver_session.results.graphics.picture.save_picture(file_name=rel_contour_pic)
        # solver_session.tui.display.save_picture(rel_contour_pic)
        print(f'Contour picture saved to {rel_contour_pic.resolve()}')

# function to create xyplots
def create_xyplot(name: str, lines: List[str]) -> None:
    print(f'Creating xyplot plot figure [{name}]...')
    default_fields = {'vel':'x-velocity', 'turb':'turb-intensity', 'tke':'turb-kinetic-energy'}
    axis, field = name.split('_')[0], default_fields[name.split('_')[-1]]
    if axis == 'y':
        pos_on_x, pos_on_y = True, False
        y_vector, z_vector = 1, 0
        x_func, y_func = 'Direction Vector', field
    elif axis == 'z':
        pos_on_x, pos_on_y = False, True
        y_vector, z_vector = 0, 1
        x_func, y_func = field, 'Direction Vector'
    else:
        raise ValueError('axis should be either y or z!')
    exsiting_xy_plots = solver_session.results.plot.xy_plot.keys()
    if name not in exsiting_xy_plots:
        solver_session.results.plot.xy_plot.create(name)
    solver_session.results.plot.xy_plot[name] = {
        'name': name,
        'options': {'node_values': True,
                    'position_on_x_axis': pos_on_x,
                    'position_on_y_axis': pos_on_y},
        'plot_direction': {'option': 'direction-vector',
                           'direction_vector': {'x_component': 0,
                                                'y_component': y_vector,
                                                'z_component': z_vector}},
        'x_axis_function': x_func,
        'y_axis_function': y_func,
        'surfaces_list': lines,
        }

    if config.surface.line.screen_shot.enable:
        solver_session.results.plot.xy_plot[name].display()
        rel_plot_pic = Path(image_folder, name + '.png')
        if rel_plot_pic.exists(): os.remove(rel_plot_pic)
        solver_session.results.graphics.picture.save_picture(file_name=rel_plot_pic)
        # solver_session.tui.display.save_picture(rel_plot_pic)
        print(f'xyplot picture saved to {rel_plot_pic.resolve()}')

if config.general.run.plot:
    if config.surface.plot.reset:
        # solver_session.results.surfaces.plane_surface.clear()
        solver_session.results.graphics.contour.clear()
        solver_session.results.plot.xy_plot.clear()

    if config.surface.plot.enable:
        # create contours in xy-plane, xz-plane and yz-plane
        for contour_name in config.surface.plot.contour_name:
            create_contour(contour_name)

        # create xyplots in y-direction and z-direction
        for profile_name, line_name in zip(
            config.surface.plot.profile_name, [y_hub_line_names] * 2 + [z_hub_line_names] * 2):
            create_xyplot(profile_name, line_name)

solver_session.results.graphics.contour.keys()
solver_session.results.plot.xy_plot.keys()

Creating contour plot figure [xy_hub_vel]...
Contour picture saved to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\image\xy_hub_vel.png
Creating contour plot figure [xy_hub_turb]...
Contour picture saved to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\image\xy_hub_turb.png
Creating contour plot figure [xz_hub_vel]...
Contour picture saved to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\image\xz_hub_vel.png
Creating contour plot figure [xz_hub_turb]...
Contour picture saved to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\image\xz_hub_turb.png
Creating xyplot plot figure [y_hub_vel]...
xyplot picture saved to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\image\y_hub_vel.png
Creating xyplot plot figure [y_hub_turb]...
xyplot picture saved to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\image\y_hub_turb.png
Creating xyplot plot figure [z_hub_vel]...
xyplot picture saved to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\image\z_hub_vel.png
Creating xyplot plo

dict_keys(['y_hub_vel', 'y_hub_turb', 'z_hub_vel', 'z_hub_turb'])

In [15]:
if config.output.case.enable:
    case_new_path = Path(case_path, config.output.case.name + '.cas')
    save_name = config.output.case.name
    print(f'Project and data saved as: {case_new_path}')
else:
    case_old_path = Path(case_path, config.general.case.name + '.cas')
    save_name = config.general.case.name
    print(f'Project and data updated: {case_old_path}')

solver_session.file.write(
    file_type='case-data',
    file_name=save_name,
    )

print('Case and data file saved done!')

Project and data updated: C:\Users\Li Hang\Documents\Projects\pyfluent\demo\test.cas
Writing "test_eq.cas"...
  207944 hexahedral cells, zone  8, binary.
  207944 cell partition ids, zone  8, 6 partitions, binary.
  611494 quadrilateral interior faces, zone  9, binary.
    1496 quadrilateral velocity-inlet faces, zone 10, binary.
    1496 quadrilateral outflow faces, zone 11, binary.
    6116 quadrilateral wall faces, zone 12, binary.
   15568 quadrilateral symmetry faces, zone 13, binary.
  220500 nodes, binary.
  220500 node flags, binary.
Done.

Writing "test_eq.dat"...
Done.

Case and data file saved done!


Copy "C:\Users\Li Hang\Documents\Projects\pyfluent\demo\inflow_YangYi.c" to "libudf_eq\\src"


Copyright 1987-2023 ANSYS, Inc. All Rights Reserved.
Compiler and linker: Clang (builtin)
Compiler path: "C:\\PROGRA~1\\ANSYSI~1\\v231\\fluent"\\ntbin\\clang\\bin\\clang-cl
Linker   path: "C:\\PROGRA~1\\ANSYSI~1\\v231\\fluent"\\ntbin\\clang\\bin\\lld-link
C sources: ['inflow_YangYi.c']

Copyright 1987-2023 ANSYS, Inc. All Rights Reserved.
Compiler and linker: Clang (builtin)
Compiler path: "C:\\PROGRA~1\\ANSYSI~1\\v231\\fluent"\\ntbin\\clang\\bin\\clang-cl
Linker   path: "C:\\PROGRA~1\\ANSYSI~1\\v231\\fluent"\\ntbin\\clang\\bin\\lld-link
C sources: ['inflow_YangYi.c']

Done.

forestriveral: Opening library "C:\\Users\\Li Hang\\Documents\\Projects\\pyfluent\\demo\\libudf_eq"...
Done.

 Standard k-e turbulence model for inflow profile (Yi Yang et al.): u_star= 0.325600, K= 0.418700, rho= 1.225000, z_0= 0.000221C_mu= 0.400000, C_1= -0.976900, C_2= 7.282900Sigma_e= 1.300000, C2e_1e= 0.880000

for

In [ ]:
if config.general.run.export:
    # output wake field in different planes and lines.
    if config.output.data.ascii.enable:
        surface_names = [xy_plane_names, xz_plane_names, yz_plane_names,
                        y_hub_line_names, z_hub_line_names]
        for name, surface in zip(ascii_data_name, surface_names):
            rel_export_path = Path(data_folder, f'{name}.txt')
            if rel_export_path.exists(): os.remove(rel_export_path)
            print(f'Outputing ASCII data to {rel_export_path.resolve()}...')
            solver_session.tui.file.export.ascii(
                f'{data_folder}/{name}.txt',
                tuple(surface),
                'yes',
                'x-velocity', 'y-velocity', 'z-velocity', 'turb-intensity', 'turb-kinetic-energy',
                '()',
                'no',
                )

    # output tecplot data for postprocessing.
    if config.output.data.tecplot.enable:
        rel_tecplot_name = Path(data_folder, tecplot_data_name + '.plt')
        if rel_tecplot_name.exists(): os.remove(rel_tecplot_name)
        print(f'Outputing Tecplot data to {rel_tecplot_name.resolve()}...')
        solver_session.tui.file.export.tecplot(
            f'{data_folder}/{tecplot_data_name}.plt',
            '()',
            'x-coordinate', 'y-coordinate', 'z-coordinate',
            'x-velocity', 'y-velocity', 'z-velocity', 'turb-intensity', 'turb-kinetic-energy',
            '()',
            )


Outputing ASCII data to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\data\xy_hub_field.txt...

Writing "./data/xy_hub_field.txt"...

Writing information to ascii file ...
Done.

Outputing ASCII data to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\data\xz_hub_field.txt...

Writing "./data/xz_hub_field.txt"...

Writing information to ascii file ...
Done.

Outputing ASCII data to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\data\yz_hub_field.txt...

Writing "./data/yz_hub_field.txt"...

Writing information to ascii file ...
Done.

Outputing ASCII data to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\data\y_hub_profile.txt...

Writing "./data/y_hub_profile.txt"...

Writing information to ascii file ...
Done.

Outputing ASCII data to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\data\z_hub_profile.txt...

Writing "./data/z_hub_profile.txt"...

Writing information to ascii file ...
Done.

Outputing Tecplot data to C:\Users\Li Hang\Documents\Projects\pyfluent\demo\data\

In [14]:
# extract wake field data from created surfaces
def surface_field_data(
    solver: pyfluent.session.BaseSession,
    surface: str,
    field_name: List[str] = config.general.field.value,
    ) -> Dict[str, Any]:
    field_data = solver.field_data
    assert isinstance(surface, str), 'Surface name should be string!'
    transaction = solver_session.field_data.new_transaction()
    transaction.add_surfaces_request(
        surface_names=[surface],
        provide_vertices=True,
        provide_faces=False,
    )
    for value in field_name:
        transaction.add_scalar_fields_request(
            surface_names=[surface],
            field_name=value,
        )

    flow_data = transaction.get_fields()
    coords, field_data = None, None
    for data_type, data in flow_data.items():
        tag = data_type[0][1]; key = list(data.keys()); # print(tag, key)
        assert len(key) == 1, 'Data should only have one key!'
        if tag == 'surface-data':
            coords = data[key[0]]['vertices'].reshape(-1, 3)
        elif tag == 'scalar-field':
            field_data = copy.deepcopy(data[key[0]])
        else:
            continue

    for value in field_name:
        assert isinstance(field_data[value], np.ndarray), \
            f'Field data {value} should be numpy array!'
        assert field_data[value].shape[0] == coords.shape[0], \
            f'Field data length {field_data[value].shape[0]} should be equal to coords length {coords.shape[0]}!'
        field_data[value] = np.round(field_data[value], 4).tolist()

    field_data['coordinate'] = np.round(coords.copy(), 4).tolist()
    return field_data

# field data output loop for all created surfaces
def field_data_output(
    solver: pyfluent.session.BaseSession,
    surfaces: List[str],
    field_name: List[str] = config.general.field.value,
    ) -> Dict[str, Any]:
    wake_data = {}
    for i, surface in enumerate(surfaces):
        print(f'[**INFO**] Processing wake data of {surface} ({i+1}/{len(surfaces)})...')
        wake_data[surface] = surface_field_data(solver, surface, field_name)

    data_save_path = Path(data_folder, wake_data_name + '.txt')
    print('[**INFO**] Saving wake data to file...')
    with open(data_save_path, "w+") as outfile:
        json.dump(wake_data, outfile, indent=4)
    return wake_data

if config.general.run.export and config.output.data.field.enable:
    surface_list = xy_plane_names + xz_plane_names + yz_plane_names \
        + y_hub_line_names + z_hub_line_names + ['z_hub_inflow']
    wake_data = field_data_output(solver_session, surface_list)
    wake_data.keys()

[**INFO**] Processing wake data of xy_hub (1/60)...
[**INFO**] Processing wake data of xz_hub (2/60)...
[**INFO**] Processing wake data of yz_-3d (3/60)...
[**INFO**] Processing wake data of yz_-2d (4/60)...
[**INFO**] Processing wake data of yz_-1d (5/60)...
[**INFO**] Processing wake data of yz_0d (6/60)...
[**INFO**] Processing wake data of yz_1d (7/60)...
[**INFO**] Processing wake data of yz_2d (8/60)...
[**INFO**] Processing wake data of yz_3d (9/60)...
[**INFO**] Processing wake data of yz_4d (10/60)...
[**INFO**] Processing wake data of yz_5d (11/60)...
[**INFO**] Processing wake data of yz_6d (12/60)...
[**INFO**] Processing wake data of yz_7d (13/60)...
[**INFO**] Processing wake data of yz_8d (14/60)...
[**INFO**] Processing wake data of yz_9d (15/60)...
[**INFO**] Processing wake data of yz_10d (16/60)...
[**INFO**] Processing wake data of yz_11d (17/60)...
[**INFO**] Processing wake data of yz_12d (18/60)...
[**INFO**] Processing wake data of yz_13d (19/60)...
[**INFO**] P

In [ ]:
if config.general.exit_at_end:
    solver_session.exit()

In [ ]:
solver_session.exit()

NameError: name 'solver_session' is not defined

In [ ]:
if config.general.clear_log_file:
    print('Checking and removing log files...')
    pattern = os.path.join(case_path, "fluent-*-error.log")
    log_files_to_remove = glob.glob(pattern)
    for file in log_files_to_remove:
        os.remove(file)
        print(f"Removed file: {file}")

Checking and removing log files...


In [ ]:
# vertices_data = field_data.get_surface_data(
#     surface_name="y_hub_3d",
#     data_type=SurfaceDataType.Vertices
#     )
# vertices_data_size = vertices_data.size
# vertices_data_surface_id = vertices_data.surface_id
# print('Data size :', vertices_data_size)
# print('Surface ID :', vertices_data_surface_id)
# # print(vertices_data[0].x, vertices_data[0].y, vertices_data[0].z)
# # print(vertices_data[-1].x, vertices_data[-1].y, vertices_data[-1].z)

# for i in range(vertices_data_size):
#     vertice = vertices_data[i]
#     print(i, vertice.x, vertice.y, vertice.z)


# scalar_field_data = field_data.get_scalar_field_data(
#     surface_name="y_hub_3d",
#     field_name="x-velocity"
#     )

# field_data_size = scalar_field_data.size
# field_data_surface_id = scalar_field_data.surface_id
# print('Data size :', field_data_size)
# print('Surface ID :', field_data_surface_id)

# for i in range(field_data_size):
#     print(i, scalar_field_data[i].scalar_data)

In [ ]:
# from ansys.fluent.visualization import set_config
# from ansys.fluent.visualization.matplotlib import Plots
# from ansys.fluent.visualization.pyvista import Graphics

# set_config(blocking=True, set_view_on_display="isometric")
# graphics = Graphics(session=solver_session)
# surf_xy_plane = graphics.Surfaces["xy-plane"]
# surf_xy_plane.definition.type = "plane-surface"
# plane_surface_xy = surf_xy_plane.definition.plane_surface
# plane_surface_xy.z = 0.125
# surf_xy_plane.display("window-1")

# mesh1 = graphics.Meshes["mesh-1"]
# mesh1.show_edges = True
# mesh1.surfaces_list = [
#     'inlet',
#     'outlet',
#     'ground',
#     'side',
#     'top',
# ]
# mesh1.display("window-2")

# plots_session_1 = Plots(solver_session)
# xy_plot = plots_session_1.XYPlots["xy-plot"]
# xy_plot.surfaces_list = ['y_hub_3d', 'y_hub_6d', 'y_hub_9d', 'y_hub_12d', 'y_hub_15d']
# xy_plot.y_axis_function = 'x-velocity'
# xy_plot.plot("window-3")